In [265]:
import pandas as pd
import os
import shutil



plan_data = pd.read_csv(r'GLO RAS Model Runs Distribution to Modelers.csv')
plan_data.head()

,GLO RAS Events,Modeler,PlanID,U Flow File ID,Geometry
0,BaseModel,NaN,p01,u01,g01
1,050yr_Atlas_14,Angshuman,p30,u30,g01
2,050y_ Left_PD_Bivariate,Angshuman,p31,u31,g01
3,050yr_MostLikely_Bivariate,Angshuman,p32,u32,g01
4,050yr_Right_SD_Bivariate,Angshuman,p33,u33,g01


In [266]:
projDir = "Z:\GLO\Models\Set_Plans\HECRASV6.3.1_BaseModel"
projTitle = 'East_Galveston_Bay'

In [267]:
# open ras_prj file and read the data.
with open(fr"{projDir}\{projTitle}.prj", 'r') as f:
    ras_prj = f.readlines()

In [268]:
# use list comprehension to remove the string '\n' from each item in the list
ras_prj = [x.strip('\n') for x in ras_prj]
# ras_prj

In [269]:
projTitle = [s.split('=')[-1] for s in ras_prj if 'Proj Title' in s][0]

In [270]:
# Filter each row to search for the word "Plan File" and return a new dataframe with those rows
find_str = 'Plan File'
# description = [s for s in ras_prj if find_str in s][0].split(":")[1:][0].strip()
plans = [s.split('=')[-1] for s in ras_prj if find_str in s]
# plans


In [271]:
# Filter each row to search for the word "Plan File" and return a new dataframe with those rows
find_str = 'Unsteady File'
# description = [s for s in ras_prj if find_str in s][0].split(":")[1:][0].strip()
u_files = [s.split('=')[-1] for s in ras_prj if find_str in s]
# plans

In [272]:
plan_titles_found = {}
for plan in plans:
    planfn = f'{projDir}\{projTitle}.{plan}'
    with open(planfn, 'r') as p:
        plan_file = p.readlines()
    plan_title = [s.split('=')[-1].split('\n')[0] for s in plan_file if 'Plan Title' in s][0]
    plan_titles_found[plan] = plan_title
pd.DataFrame.from_dict(plan_titles_found, orient='index', columns=['Plan Title'])

,Plan Title
p01,BaseModel
p10,010yr_Atlas_14
p11,010yr_Left_PD_Bivariate
p12,010yr_MostLikely_Bivariate
p13,010yr_Right_SD_Bivariate
p30,050yr_Atlas_14
p31,050y_ Left_PD_Bivariate
p32,050yr_MostLikely_Bivariate
p33,050yr_Right_SD_Bivariate
p50,100yr_Atlas_14


In [273]:
# clean out output dir
import glob
for f in glob.glob('./new_files/*'):
    os.remove(f)

# create a list of new_planIDs
new_planIDs = []
new_uIDs = []

# For each plan, open the corresponding file and extract the data
for plan in plans:
    planfn = f'{projDir}\{projTitle}.{plan}'
    with open(planfn, 'r') as p:
        plan_file = p.readlines()
    plan_title = [s.split('=')[-1].split('\n')[0] for s in plan_file if 'Plan Title' in s][0]
    # Find plan_title in plan_data dataframe. Get the PlanID column.
    wanted_planID = plan_data[plan_data['GLO RAS Events'] == plan_title]['PlanID'].values[0]
    new_planIDs.append(wanted_planID)
    # For testing, Check if all rows have a matching event name in the dataframe vs the plan title.
    # x = plan_data[plan_data['GLO RAS Events'] == plan_title]['PlanID']
    # if len(x) == 0:
    #     print(f'{plan_title} not found in plan dataframe')

    # Create new u and u hdf files using the wanted planID.
    uID = plan.replace('p','u')
    u_hdfID = uID+'.hdf'
    wanted_uID = wanted_planID.replace('p','u')
    new_uIDs.append(wanted_uID)
    wanted_u_hdfID = wanted_uID+'.hdf'
    shutil.copyfile(os.path.join(projDir,f'{projTitle}.{uID}'), os.path.join(os.getcwd(),'old_files', f'{projTitle}.{uID}'))
    os.rename(f'./old_files/{projTitle}.{uID}', f'./new_files/{projTitle}.{wanted_uID}')
    shutil.copyfile(os.path.join(projDir,f'{projTitle}.{u_hdfID}'), os.path.join(os.getcwd(),'old_files', f'{projTitle}.{u_hdfID}'))
    os.rename(f'./old_files/{projTitle}.{u_hdfID}', f'./new_files/{projTitle}.{wanted_u_hdfID}')

    u_in_plan = [s.split('=')[-1].split('\n')[0] for s in plan_file if 'Flow File=' in s][0]

    # Create new plan files using the wanted planID.
    # shutil.copyfile(os.path.join(projDir,f'{projTitle}.{plan}'), os.path.join(os.getcwd(),'old_files', f'{projTitle}.{plan}'))
    # os.rename(f'./old_files/{projTitle}.{plan}', f'./new_files/{projTitle}.{wanted_planID}')
    
    # Replace flow file ID in plan file with wanted_uID.
    u_index_in_plan = [i for i,s in enumerate(plan_file) if 'Flow File=' in s][0]
    plan_file[u_index_in_plan] = f'Flow File={wanted_uID}\n'
    # Create new plan files using the wanted planID.
    with open(f'./new_files/{projTitle}.{plan}', 'w') as fout:
        fout.writelines(plan_file)
    

In [ ]:
u_index_in_plan = [i for i,s in enumerate(plan_file) if 'Flow File=' in s][0]
plan_file[u_index_in_plan] = f'Flow File={wanted_uID}\n'


In [ ]:
# update the plan IDs in the prj file
shutil.copyfile(os.path.join(projDir,f'{projTitle}.prj'), os.path.join(os.getcwd(), f'{projTitle}.prj'))
with open(f'{projTitle}.prj', 'r') as prj:
    new_ras_prj = prj.readlines()
new_ras_prj = [x.strip('\n') for x in new_ras_prj]
# Filter each row to search for the word "Plan File" and remove those indexes from the list, append the new plan IDs to the list.
find_str = 'Plan File'
plan_indexes = [i for i,s in enumerate(ras_prj) if find_str in s]
for index in sorted(plan_indexes, reverse=True):
    del new_ras_prj[index]
for plan in new_planIDs:
    new_ras_prj.append(f'Plan File={plan}')

# Filter each row to search for the word "Unsteady File" and remove those indexes from the list, append the new plan IDs to the list.
find_str = 'Unsteady File'
unsteady_indexes = [i for i,s in enumerate(ras_prj) if find_str in s]
for index in sorted(unsteady_indexes, reverse=True):
    del new_ras_prj[index]
for u in new_uIDs:
    new_ras_prj.append(f'Unsteady File={u}')


In [ ]:
# add '\n' to each line
new_ras_prj = [s+'\n' for s in new_ras_prj]

In [ ]:
 with open(f'new_{projTitle}.prj', 'w') as fout:
    fout.writelines(new_ras_prj)